In [1]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
import xgboost as xgb
    

In [2]:
df = pd.read_csv("GLRLM_cyto.csv")
df

,0.0011806,0.0012137,0.001208,0.0012018,1071.2,1046.8,1048.8,1055,1180.7,1552.3,...,0.0046729,778.89,795.98,788.88,839.12,4806.8,3260.5,3810,2836.1,-1
0,0.001107,0.001119,0.001116,0.001109,1039.50,1033.20,1037.90,1042.00,2061.90,2842.8,...,0.004115,690.72,744.43,709.54,766.11,5627.8,3720.7,5336.2,3671.1,-1
1,0.000838,0.000817,0.000813,0.000816,1474.20,1506.50,1512.10,1509.20,3152.70,4917.1,...,0.003754,871.63,1030.40,1000.50,1020.10,13398.0,7915.7,11760.0,8232.1,-1
2,0.000959,0.000957,0.000959,0.000958,1173.50,1181.10,1182.60,1181.20,989.97,1348.5,...,0.003669,761.77,836.36,800.78,846.97,7517.2,4942.6,6486.0,4416.6,-1
3,0.001199,0.001197,0.001212,0.001218,998.09,1001.80,989.75,984.42,930.80,1843.0,...,0.004436,641.78,785.93,751.50,760.29,6421.8,2951.4,3344.0,2882.6,-1
4,0.001475,0.001478,0.001462,0.001484,907.20,910.03,921.85,900.22,955.55,1267.9,...,0.008193,665.58,725.56,684.40,712.31,4015.4,2846.9,3709.8,2642.7,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,0.001524,0.001544,0.001554,0.001533,771.82,765.06,760.72,767.49,1757.30,2667.9,...,0.009052,449.22,505.74,481.86,525.14,6456.4,4333.4,5510.5,3652.4,-1
912,0.002081,0.002125,0.002143,0.002136,717.41,706.27,692.82,697.34,1442.90,1588.2,...,0.008687,605.75,603.61,527.33,604.96,1930.2,1742.7,2547.4,1486.1,-1
913,0.001254,0.001253,0.001270,0.001281,1184.30,1209.10,1198.60,1190.80,2033.90,2822.1,...,0.004550,829.15,925.68,873.47,902.69,6960.8,4208.7,5503.9,4354.0,-1
914,0.001481,0.001492,0.001491,0.001518,888.70,883.39,877.48,866.00,1548.80,2111.8,...,0.004107,682.09,734.07,688.15,697.72,3022.3,2036.6,2609.3,2124.5,-1


## Trocando a classe saúdavel de -1 para 0

In [3]:
y = df.iloc[:, -1]
y = y.map({-1: 0, 1: 1})
y

0      0
1      0
2      0
3      0
4      0
      ..
911    0
912    0
913    0
914    0
915    0
Name: -1, Length: 916, dtype: int64

In [4]:
y.value_counts()

-1
0    674
1    242
Name: count, dtype: int64

In [5]:
X = df.drop(columns=df.columns[-1])
X

,0.0011806,0.0012137,0.001208,0.0012018,1071.2,1046.8,1048.8,1055,1180.7,1552.3,...,0.0061704,0.0046729,778.89,795.98,788.88,839.12,4806.8,3260.5,3810,2836.1
0,0.001107,0.001119,0.001116,0.001109,1039.50,1033.20,1037.90,1042.00,2061.90,2842.8,...,0.005960,0.004115,690.72,744.43,709.54,766.11,5627.8,3720.7,5336.2,3671.1
1,0.000838,0.000817,0.000813,0.000816,1474.20,1506.50,1512.10,1509.20,3152.70,4917.1,...,0.005271,0.003754,871.63,1030.40,1000.50,1020.10,13398.0,7915.7,11760.0,8232.1
2,0.000959,0.000957,0.000959,0.000958,1173.50,1181.10,1182.60,1181.20,989.97,1348.5,...,0.005446,0.003669,761.77,836.36,800.78,846.97,7517.2,4942.6,6486.0,4416.6
3,0.001199,0.001197,0.001212,0.001218,998.09,1001.80,989.75,984.42,930.80,1843.0,...,0.005279,0.004436,641.78,785.93,751.50,760.29,6421.8,2951.4,3344.0,2882.6
4,0.001475,0.001478,0.001462,0.001484,907.20,910.03,921.85,900.22,955.55,1267.9,...,0.012025,0.008193,665.58,725.56,684.40,712.31,4015.4,2846.9,3709.8,2642.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,0.001524,0.001544,0.001554,0.001533,771.82,765.06,760.72,767.49,1757.30,2667.9,...,0.013590,0.009052,449.22,505.74,481.86,525.14,6456.4,4333.4,5510.5,3652.4
912,0.002081,0.002125,0.002143,0.002136,717.41,706.27,692.82,697.34,1442.90,1588.2,...,0.015723,0.008687,605.75,603.61,527.33,604.96,1930.2,1742.7,2547.4,1486.1
913,0.001254,0.001253,0.001270,0.001281,1184.30,1209.10,1198.60,1190.80,2033.90,2822.1,...,0.006287,0.004550,829.15,925.68,873.47,902.69,6960.8,4208.7,5503.9,4354.0
914,0.001481,0.001492,0.001491,0.001518,888.70,883.39,877.48,866.00,1548.80,2111.8,...,0.005301,0.004107,682.09,734.07,688.15,697.72,3022.3,2036.6,2609.3,2124.5


In [6]:
kfold = KFold(n_splits=5, shuffle=True, random_state=19)

## testando com Kfold

In [7]:
for i, (train_index, test_index) in enumerate(kfold.split(X)):

    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]

    #smote = SMOTE(random_state=19)
    #x_train, y_train = smote.fit_resample(x_train, y_train)

    smote_enn = SMOTEENN(random_state=19) 
    x_train, y_train = smote_enn.fit_resample(x_train, y_train) #aplicando o balanceamento dos dados no treinamento

    model = xgb.XGBClassifier(n_estimators=1000,max_depth=3,colsample_bytree=1.0,learning_rate=0.2,subsample=1.0, scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train))

    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    kappa = cohen_kappa_score(y_test, y_pred)

    print(f'Fold Kappa: {kappa}')

"""

resultados por teste

normal

Fold Kappa: 0.5939849624060151
Fold Kappa: 0.6218407248450166
Fold Kappa: 0.5794535705166397
Fold Kappa: 0.7283254156769596
Fold Kappa: 0.5813725490196079

smote

Fold Kappa: 0.7266187050359711
Fold Kappa: 0.5768628328485119
Fold Kappa: 0.5696212797015652
Fold Kappa: 0.6694915254237288
Fold Kappa: 0.5991530373831775

smote_enn

Fold Kappa: 0.631578947368421
Fold Kappa: 0.43423271500843175
Fold Kappa: 0.5459739869182768
Fold Kappa: 0.5221932114882507
Fold Kappa: 0.5574679943100995


"""

Fold Kappa: 0.7027833001988071
Fold Kappa: 0.5293768545994065
Fold Kappa: 0.5459057071960298
Fold Kappa: 0.5125055498002072
Fold Kappa: 0.4851437590063816


'\n\nresultados por teste\n\nnormal\n\nFold Kappa: 0.5939849624060151\nFold Kappa: 0.6218407248450166\nFold Kappa: 0.5794535705166397\nFold Kappa: 0.7283254156769596\nFold Kappa: 0.5813725490196079\n\nsmote\n\nFold Kappa: 0.7266187050359711\nFold Kappa: 0.5768628328485119\nFold Kappa: 0.5696212797015652\nFold Kappa: 0.6694915254237288\nFold Kappa: 0.5991530373831775\n\nsmote_enn\n\nFold Kappa: 0.631578947368421\nFold Kappa: 0.43423271500843175\nFold Kappa: 0.5459739869182768\nFold Kappa: 0.5221932114882507\nFold Kappa: 0.5574679943100995\n\n\n'

In [8]:

# Aumentando os dados antes de terinando para balancear
df = pd.concat([X, y], axis=1)
df.columns = list(X.columns) + ['target']  # coluna alvo

X_smote, y_smote = SMOTE(random_state=19).fit_resample(df.drop(columns=['target']), df['target'])

df_balanced = pd.concat([X_smote, y_smote], axis=1)

X_balanced = df_balanced.drop(columns=['target'])
y_balanced = df_balanced['target']

best = 0

for i, (train_index, test_index) in enumerate(kfold.split(X)):

    x_train, x_test = np.array(X_balanced)[train_index], np.array(X_balanced)[test_index]
    y_train, y_test = np.array(y_balanced)[train_index], np.array(y_balanced)[test_index]

    #smote = SMOTE(random_state=19)
    #x_train, y_train = smote.fit_resample(x_train, y_train)

    model = xgb.XGBClassifier(
        learning_rate=0.1, 
        n_estimators=1000, 
        max_depth=10,
        min_child_weight=1,
        gamma=0,
        subsample=1,
        scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train)
        )

    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    kappa = cohen_kappa_score(y_test, y_pred)

    if kappa > best:
        best = kappa

        best_model = model

    print(f'Fold Kappa: {kappa}')


"""

learning_rate = None
n_stimators = None

Fold Kappa: 0.6240601503759399
Fold Kappa: 0.5400565504241281
Fold Kappa: 0.5650911011354633
Fold Kappa: 0.7729694227541415
Fold Kappa: 0.5377231391040755

learning_rate = 0.1

Fold Kappa: 0.6642335766423357
Fold Kappa: 0.5400565504241281
Fold Kappa: 0.5775923295454546
Fold Kappa: 0.7373331419549304
Fold Kappa: 0.5685415964971371

learning_rate = 0.1
n_estimators = 1000

Fold Kappa: 0.6541353383458647
Fold Kappa: 0.6049412329095706
Fold Kappa: 0.5723929035049762
Fold Kappa: 0.7729694227541415
Fold Kappa: 0.617698671346202

learning_rate=0.1
n_estimators=1000
max_depth=10

Fold Kappa: 0.6417910447761195
Fold Kappa: 0.546414008155433
Fold Kappa: 0.5903107369929256
Fold Kappa: 0.7875181422351234
Fold Kappa: 0.5932998600707877


learning_rate=0.1, 
n_estimators=1000, 
max_depth=10,
min_child_weight=1,

Fold Kappa: 0.6417910447761195
Fold Kappa: 0.546414008155433
Fold Kappa: 0.5903107369929256
Fold Kappa: 0.7875181422351234
Fold Kappa: 0.5932998600707877


learning_rate=0.1, 
n_estimators=1000, 
max_depth=10,
min_child_weight=1,
gamma=0,

Fold Kappa: 0.6417910447761195
Fold Kappa: 0.546414008155433
Fold Kappa: 0.5903107369929256
Fold Kappa: 0.7875181422351234
Fold Kappa: 0.5932998600707877


learning_rate=0.1, 
n_estimators=1000, 
max_depth=10,
min_child_weight=1,
gamma=0,
subsample=1,

Fold Kappa: 0.6417910447761195
Fold Kappa: 0.546414008155433
Fold Kappa: 0.5903107369929256
Fold Kappa: 0.7875181422351234
Fold Kappa: 0.5932998600707877

"""


Fold Kappa: 0.6983606557377049
Fold Kappa: 0.5936135998162186
Fold Kappa: 0.6274292059966685
Fold Kappa: 0.617698671346202
Fold Kappa: 0.659217877094972


'\n\nlearning_rate = None\nn_stimators = None\n\nFold Kappa: 0.6240601503759399\nFold Kappa: 0.5400565504241281\nFold Kappa: 0.5650911011354633\nFold Kappa: 0.7729694227541415\nFold Kappa: 0.5377231391040755\n\nlearning_rate = 0.1\n\nFold Kappa: 0.6642335766423357\nFold Kappa: 0.5400565504241281\nFold Kappa: 0.5775923295454546\nFold Kappa: 0.7373331419549304\nFold Kappa: 0.5685415964971371\n\nlearning_rate = 0.1\nn_estimators = 1000\n\nFold Kappa: 0.6541353383458647\nFold Kappa: 0.6049412329095706\nFold Kappa: 0.5723929035049762\nFold Kappa: 0.7729694227541415\nFold Kappa: 0.617698671346202\n\nlearning_rate=0.1\nn_estimators=1000\nmax_depth=10\n\nFold Kappa: 0.6417910447761195\nFold Kappa: 0.546414008155433\nFold Kappa: 0.5903107369929256\nFold Kappa: 0.7875181422351234\nFold Kappa: 0.5932998600707877\n\n\nlearning_rate=0.1, \nn_estimators=1000, \nmax_depth=10,\nmin_child_weight=1,\n\nFold Kappa: 0.6417910447761195\nFold Kappa: 0.546414008155433\nFold Kappa: 0.5903107369929256\nFold Ka

In [9]:
with open("xgboost_model.pkl", "wb") as file:
    pickle.dump(best_model, file)